In [ ]:
!pip install kaggle


In [ ]:
!mkdir ~/.kaggle


In [ ]:
!cp kaggle.json ~/.kaggle/


cp: cannot stat 'kaggle.json': No such file or directory


In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download devgovindani/butterfly-multiclass-image-classification-cnn


403 - Forbidden - Permission 'datasets.get' was denied


In [ ]:
!kaggle datasets download pnkjgpt/butterfly-classification-dataset

Dataset URL: https://www.kaggle.com/datasets/pnkjgpt/butterfly-classification-dataset
License(s): CC0-1.0
100% 425M/426M [00:12<00:00, 43.2MB/s]
100% 426M/426M [00:12<00:00, 35.6MB/s]


In [ ]:
!unzip butterfly-classification-dataset.zip -d butterfly-dataset

Archive:  butterfly-classification-dataset.zip
  inflating: butterfly-dataset/Train/adonis/000.jpg  
  inflating: butterfly-dataset/Train/adonis/001.jpg  
  inflating: butterfly-dataset/Train/adonis/002.jpg  
  inflating: butterfly-dataset/Train/adonis/003.jpg  
  inflating: butterfly-dataset/Train/adonis/004.jpg  
  inflating: butterfly-dataset/Train/adonis/005.jpg  
  inflating: butterfly-dataset/Train/adonis/006.jpg  
  inflating: butterfly-dataset/Train/adonis/007.jpg  
  inflating: butterfly-dataset/Train/adonis/008.jpg  
  inflating: butterfly-dataset/Train/adonis/009.jpg  
  inflating: butterfly-dataset/Train/adonis/010.jpg  
  inflating: butterfly-dataset/Train/adonis/011.jpg  
  inflating: butterfly-dataset/Train/adonis/012.jpg  
  inflating: butterfly-dataset/Train/adonis/013.jpg  
  inflating: butterfly-dataset/Train/adonis/014.jpg  
  inflating: butterfly-dataset/Train/adonis/015.jpg  
  inflating: butterfly-dataset/Train/adonis/016.jpg  
  inflating: butterfly-dataset/Trai

In [ ]:
import pandas as pd
from pathlib import Path
import os

# Set the dataset root path
DATASET_ROOT = Path("/content/butterfly-dataset/Train")

def create_butterfly_dataframe():
    """Create a DataFrame from the butterfly dataset directory structure"""
    image_files = []
    labels = []

    # Walk through all subdirectories
    for class_folder in DATASET_ROOT.iterdir():
        if class_folder.is_dir():  # Make sure it's a directory
            class_name = class_folder.name
            for img_path in class_folder.glob('*.*'):  # Get all files
                if img_path.suffix.lower() in ['.jpg', '.jpeg', '.png']:  # Only image files
                    # Store relative path from DATASET_ROOT
                    relative_path = img_path.relative_to(DATASET_ROOT)
                    image_files.append(str(relative_path))
                    labels.append(class_name)

    # Create DataFrame
    df = pd.DataFrame({
        'filename': image_files,
        'label': labels
    })
    return df

# Create and display the DataFrame
train_df = create_butterfly_dataframe()
print("DataFrame loaded successfully. Check contents:")
print(f"\nTotal images: {len(train_df)}")
print(f"Unique classes: {len(train_df['label'].unique())}")
print("\nFirst few rows:")
print(train_df.head())

# Optional: Save the DataFrame to CSV
# train_df.to_csv(DATASET_ROOT / "training_set.csv", index=False)


DataFrame loaded successfully. Check contents:

Total images: 4479
Unique classes: 50

First few rows:
               filename         label
0  large marble/016.jpg  large marble
1  large marble/062.jpg  large marble
2  large marble/042.jpg  large marble
3  large marble/009.jpg  large marble
4  large marble/054.jpg  large marble


In [ ]:
import torch.nn as nn

class VanillaCNN(nn.Module):
    def __init__(self,input_dims=3,num_classes=50,dropout=0.):
        super().__init__()
        self.block_0=nn.Sequential(
            nn.Conv2d(input_dims,64,3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.block_1=nn.Sequential(
            nn.Conv2d(64,256,3),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.block_2=nn.Sequential(
            nn.Conv2d(256,128,3),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.block_3=nn.Sequential(
            nn.Conv2d(128,64,3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.block_4=nn.Sequential(
            nn.Conv2d(64,32,3),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.block_5=nn.Sequential(
            nn.Conv2d(32,32,3),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten()
        )
        self.head=nn.Sequential(
            nn.Linear(32,128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(128,64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Linear(64,num_classes)
        )

    def forward(self, x=None ,pretrained=None):
        x = self.block_0(x)
        x = self.block_1(x)
        x = self.block_2(x)

        x = self.block_3(x)

        x = self.block_4(x)

        x = self.block_5(x)

        x= self.head(x)
        return x



In [ ]:

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from PIL import Image
from tqdm import tqdm
import time
from datetime import datetime
from torchmetrics import Precision, Recall, F1Score
import matplotlib.pyplot as plt

class JPEGImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}
        self.images = []
        self.labels = []

        # Only collect JPEG images
        valid_extensions = ('.jpg', '.jpeg')
        print("Loading JPEG images...")
        for class_name in tqdm(self.classes):
            class_dir = os.path.join(root_dir, class_name)
            class_idx = self.class_to_idx[class_name]
            for img_name in os.listdir(class_dir):
                if img_name.lower().endswith(valid_extensions):
                    self.images.append(os.path.join(class_dir, img_name))
                    self.labels.append(class_idx)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

def train_vit_base():
    # Create timestamp for logging
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    log_file = f'training_results_vit_base_{timestamp}.txt'

    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Define transforms
    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.230, 0.226, 0.224])
    ])

    # Load dataset
    data_path = "/content/butterfly-dataset/Train"
    try:
        # Create custom dataset
        dataset = JPEGImageDataset(root_dir=data_path, transform=transform)
        print(f"Found {len(dataset)} JPEG images in {len(dataset.classes)} classes")

        # Initialize logging
        with open(log_file, 'w') as f:
            f.write(f"Training started at: {timestamp}\n")
            f.write(f"Model: ViT-Base\n")
            f.write(f"Total images: {len(dataset)}\n")
            f.write(f"Number of classes: {len(dataset.classes)}\n")
            f.write("="*50 + "\n\n")

        # Split dataset
        train_size = int(0.8 * len(dataset))
        valid_size = len(dataset) - train_size
        train_dataset, valid_dataset = torch.utils.data.random_split(
            dataset, [train_size, valid_size]
        )

        # Create data loaders
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
        valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

        # Initialize ViT-Base model
        model = models.vit_b_16(pretrained=True)
        num_ftrs = model.heads[0].in_features
        model.heads = nn.Linear(num_ftrs, len(dataset.classes))
        model = model.to(device)

        # Initialize metrics
        precision = Precision(task="multiclass", num_classes=len(dataset.classes), average='macro')
        recall = Recall(task="multiclass", num_classes=len(dataset.classes), average='macro')
        f1 = F1Score(task="multiclass", num_classes=len(dataset.classes), average='macro')

        # Training setup
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)

        # Training loop
        num_epochs = 20
        best_valid_loss = float("inf")

        # Store metrics
        metrics = {
            'train_loss': [], 'train_acc': [],
            'valid_loss': [], 'valid_acc': [],
            'train_precision': [], 'train_recall': [], 'train_f1': [],
            'valid_precision': [], 'valid_recall': [], 'valid_f1': []
        }

        for epoch in range(num_epochs):
            epoch_start_time = time.time()

            # Training phase
            model.train()
            train_loss = 0
            correct_train = 0
            total_train = 0
            predicted_train = []
            labels_train = []

            # Progress bar for training
            with tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}") as pbar:
                for inputs, labels in pbar:
                    inputs, labels = inputs.to(device), labels.to(device)
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()

                    train_loss += loss.item()
                    _, predicted = torch.max(outputs, 1)
                    correct_train += (predicted == labels).sum().item()
                    total_train += labels.size(0)
                    predicted_train.extend(predicted.cpu().numpy())
                    labels_train.extend(labels.cpu().numpy())

                    # Update progress bar
                    pbar.set_postfix({
                        'loss': f"{train_loss/len(train_loader):.4f}",
                        'acc': f"{100.*correct_train/total_train:.2f}%"
                    })

            # Validation phase
            model.eval()
            valid_loss = 0
            correct_valid = 0
            total_valid = 0
            predicted_valid = []
            labels_valid = []

            with torch.no_grad():
                for inputs, labels in tqdm(valid_loader, desc="Validation"):
                    inputs, labels = inputs.to(device), labels.to(device)
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    valid_loss += loss.item()

                    _, predicted = torch.max(outputs, 1)
                    correct_valid += (predicted == labels).sum().item()
                    total_valid += labels.size(0)
                    predicted_valid.extend(predicted.cpu().numpy())
                    labels_valid.extend(labels.cpu().numpy())

            # Calculate metrics
            train_loss = train_loss/len(train_loader)
            valid_loss = valid_loss/len(valid_loader)
            train_acc = 100.*correct_train/total_train
            valid_acc = 100.*correct_valid/total_valid

            # Calculate precision, recall, and F1 score
            train_precision = precision(torch.tensor(predicted_train), torch.tensor(labels_train))
            train_recall = recall(torch.tensor(predicted_train), torch.tensor(labels_train))
            train_f1 = f1(torch.tensor(predicted_train), torch.tensor(labels_train))
            valid_precision = precision(torch.tensor(predicted_valid), torch.tensor(labels_valid))
            valid_recall = recall(torch.tensor(predicted_valid), torch.tensor(labels_valid))
            valid_f1 = f1(torch.tensor(predicted_valid), torch.tensor(labels_valid))

            epoch_time = time.time() - epoch_start_time

            # Store metrics
            metrics['train_loss'].append(train_loss)
            metrics['train_acc'].append(train_acc)
            metrics['valid_loss'].append(valid_loss)
            metrics['valid_acc'].append(valid_acc)
            metrics['train_precision'].append(train_precision.item())
            metrics['train_recall'].append(train_recall.item())
            metrics['train_f1'].append(train_f1.item())
            metrics['valid_precision'].append(valid_precision.item())
            metrics['valid_recall'].append(valid_recall.item())
            metrics['valid_f1'].append(valid_f1.item())

            # Print detailed results
            print(f"\n{'='*50}")
            print(f"Epoch {epoch+1}/{num_epochs} Results:")
            print(f"{'='*50}")
            print(f"\nTraining:")
            print(f" Loss: {train_loss:.4f}")
            print(f" Accuracy: {train_acc:.2f}%")
            print(f" Precision: {train_precision:.4f}")
            print(f" Recall: {train_recall:.4f}")
            print(f" F1 Score: {train_f1:.4f}")
            print(f"\nValidation:")
            print(f" Loss: {valid_loss:.4f}")
            print(f" Accuracy: {valid_acc:.2f}%")
            print(f" Precision: {valid_precision:.4f}")
            print(f" Recall: {valid_recall:.4f}")
            print(f" F1 Score: {valid_f1:.4f}")
            print(f"\nTime taken: {epoch_time:.2f} seconds")

            # Log results
            with open(log_file, 'a') as f:
                f.write(f"\nEpoch {epoch+1}/{num_epochs}\n")
                f.write(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.2f}%\n")
                f.write(f"Train Precision: {train_precision:.4f}, Train Recall: {train_recall:.4f}, Train F1 Score: {train_f1:.4f}\n")
                f.write(f"Valid Loss: {valid_loss:.4f}, Valid Accuracy: {valid_acc:.2f}%\n")
                f.write(f"Valid Precision: {valid_precision:.4f}, Valid Recall: {valid_recall:.4f}, Valid F1 Score: {valid_f1:.4f}\n")
                f.write(f"Epoch time: {epoch_time:.2f} seconds\n")

        # Save final model
        torch.save(model.state_dict(), 'vit_base_model.pth')

        # Plot and save metrics
        plt.figure(figsize=(10, 6))
        plt.plot(metrics['train_loss'], label="Train Loss")
        plt.plot(metrics['valid_loss'], label="Validation Loss")
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.savefig(f"{timestamp}_loss_plot.png")

        plt.figure(figsize=(10, 6))
        plt.plot(metrics['train_acc'], label="Train Accuracy")
        plt.plot(metrics['valid_acc'], label="Validation Accuracy")
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy (%)')
        plt.legend()
        plt.savefig(f"{timestamp}_accuracy_plot.png")

    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    train_vit_base()


ModuleNotFoundError: No module named 'torchmetrics'

In [ ]:
!pip install torchmetrics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.6/890.6 kB 30.3 MB/s eta 0:00:00
